In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

events_schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("price", DoubleType(), True)
])

In [0]:
spark.sql("""
CREATE VOLUME if not exists workspace.ecommerce.stream_volume
""")

In [0]:
spark.sql("SHOW VOLUMES IN workspace.ecommerce").show()

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

events_schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("price", DoubleType(), True)
])

In [0]:
stream_df = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .schema(events_schema) \
    .load("/Volumes/workspace/ecommerce/stream_volume/")

In [0]:
stream_df.printSchema()

In [0]:
stream_query = stream_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/Volumes/workspace/ecommerce/stream_volume/checkpoints/bronze") \
    .trigger(availableNow=True) \
    .toTable("workspace.ecommerce.events_delta")

In [0]:
spark.sql("""
SELECT event_type, COUNT(*) 
FROM workspace.ecommerce.events_delta
GROUP BY event_type
""").show()